In [2]:
import nfl_data_py as nfl
from pyspark.sql import SparkSession
from sqlalchemy import distinct
from ucimlrepo import fetch_ucirepo
from pyspark.sql import functions as F
from pyspark.sql import Window
import numpy as np
import os
print(f'{os.getcwd()}')

/home/jdwolfe/Spark-Projects/spark-learning-notebooks


In [3]:
spark = SparkSession.builder.appName("NFL-Jupyter").getOrCreate()

spark

25/05/13 13:11:49 WARN Utils: Your hostname, JordansGaming resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/13 13:11:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/13 13:11:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/13 13:11:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
years = list(np.arange(1999, 2025))
nfl_data = nfl.import_pbp_data(years, downcast=False, cache=False, alt_path=None)


print(nfl_data)

nfl_data.to_csv(f'{os.getcwd()}/data/nfl_data/full_nfl_data_pbp_99_to_24.csv', index=False)

1999 done.
2000 done.
2001 done.
2002 done.
2003 done.
2004 done.
2005 done.
2006 done.
2007 done.
2008 done.
2009 done.
2010 done.
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
         play_id          game_id old_game_id home_team away_team season_type  \
0           35.0  1999_01_ARI_PHI  1999091200       PHI       ARI         REG   
1           60.0  1999_01_ARI_PHI  1999091200       PHI       ARI         REG   
2           82.0  1999_01_ARI_PHI  1999091200       PHI       ARI         REG   
3          103.0  1999_01_ARI_PHI  1999091200       PHI       ARI         REG   
4          126.0  1999_01_ARI_PHI  1999091200       PHI       ARI         REG   
...          ...              ...         ...       ...       ...         ...   
1230850   4158.0   2024_22_KC_PHI  2025020900       PHI        KC        POST   
1230851   4181.0   2024_22_KC_PHI  2025020900       PHI        KC 

In [5]:
spark_nfl_df = spark.read.csv(f'{os.getcwd()}/data/nfl_data/full_nfl_data_pbp_99_to_24.csv', header=True, inferSchema=True).toDF(*nfl_data.columns)

spark_nfl_df.show()

25/05/13 15:01:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------------+-----------+---------+---------+-----------+----+-------+------------+-------+-------------+------------+----------+-------------------------+----------------------+----------------------+---------+-----------+-----+---+---+----+----------+-------------------+------+-------+------+--------------------+-----------+------------+-------+---------+-----------+--------+--------+-----------+-----------+-------------+---------+-----------------+------------+-------+-----------------+-------------+------------------+---------------------+-----------------------+-----------------------+-------+------------+-------+--------------+------------+--------------------------+--------------------------+----------------+----------------+-------------+-------------+------------------+------------------+------------------+-----------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------

In [16]:
spark_nfl_df = spark_nfl_df.withColumn("year", F.year(spark_nfl_df.game_date))
spark_nfl_df = spark_nfl_df.withColumn("season", F.substring(F.col('game_id'), 1, 4))

In [18]:
spark_nfl_df.printSchema()

root
 |-- play_id: double (nullable = true)
 |-- game_id: string (nullable = true)
 |-- old_game_id: integer (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- season_type: string (nullable = true)
 |-- week: integer (nullable = true)
 |-- posteam: string (nullable = true)
 |-- posteam_type: string (nullable = true)
 |-- defteam: string (nullable = true)
 |-- side_of_field: string (nullable = true)
 |-- yardline_100: double (nullable = true)
 |-- game_date: date (nullable = true)
 |-- quarter_seconds_remaining: double (nullable = true)
 |-- half_seconds_remaining: double (nullable = true)
 |-- game_seconds_remaining: double (nullable = true)
 |-- game_half: string (nullable = true)
 |-- quarter_end: double (nullable = true)
 |-- drive: double (nullable = true)
 |-- sp: double (nullable = true)
 |-- qtr: double (nullable = true)
 |-- down: double (nullable = true)
 |-- goal_to_go: double (nullable = true)
 |-- time: timestamp (nullab

In [7]:
spark_nfl_df.count()

1230855

In [19]:
spark_nfl_df \
            .select('season_type') \
            .distinct().show()

+-----------+
|season_type|
+-----------+
|       POST|
|        REG|
+-----------+



In [26]:
teams_df = spark_nfl_df \
            .select('home_team') \
            .distinct()

teams_df.show(teams_df.count())

+---------+
|home_team|
+---------+
|      NYJ|
|      CAR|
|       LA|
|       TB|
|      DET|
|      TEN|
|      BUF|
|      BAL|
|      LAC|
|       NE|
|       GB|
|      DEN|
|       SF|
|      ARI|
|       KC|
|      SEA|
|      CIN|
|      DAL|
|      CLE|
|      MIA|
|      MIN|
|       LV|
|      ATL|
|      PHI|
|      WAS|
|      NYG|
|      PIT|
|       NO|
|      IND|
|      JAX|
|      CHI|
|      HOU|
+---------+



### *What is a receiver on the Vikings with a 10+ Rec TD Season*

In [31]:
spark_nfl_df \
        .filter(F.col('season') >= 2010) \
        .show()

+-------+---------------+-----------+---------+---------+-----------+----+-------+------------+-------+-------------+------------+----------+-------------------------+----------------------+----------------------+---------+-----------+-----+---+---+----+----------+-------------------+------+-------+------+--------------------+----------+------------+-------+---------+-----------+--------+--------+-----------+-----------+-------------+---------+-----------------+------------+-------+-----------------+-------------+------------------+---------------------+-----------------------+-----------------------+-------+------------+-------+--------------+------------+--------------------------+--------------------------+----------------+----------------+-------------+-------------+------------------+------------------+------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------

In [28]:
pass_plays_df = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('td_team').isNotNull()) \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('td_team') == 'MIN') \
                .groupby('td_player_name', 'td_player_id', 'season') \
                .agg(
                F.count('*').alias('Total Touchdowns')
                )\
                .filter(F.col('Total Touchdowns') >= 10) \
                .sort('Total Touchdowns', ascending=False) \
                .show()

+--------------+------------+------+----------------+
|td_player_name|td_player_id|season|Total Touchdowns|
+--------------+------------+------+----------------+
|       R. Moss|  00-0011754|  2003|              17|
|        R.Moss|  00-0011754|  2000|              15|
|     A.Thielen|  00-0030035|  2020|              14|
|        R.Moss|  00-0011754|  2004|              13|
|      C.Carter|  00-0002721|  1999|              12|
|        R.Moss|  00-0011754|  1999|              11|
|    V.Shiancoe|  00-0022054|  2009|              11|
|        R.Moss|  00-0011754|  2001|              10|
|     A.Thielen|  00-0030035|  2021|              10|
|   J.Jefferson|  00-0036322|  2021|              10|
|     J.Addison|  00-0038994|  2023|              10|
|   J.Jefferson|  00-0036322|  2024|              10|
+--------------+------------+------+----------------+



### *What is a receiver on the GB Packers with a 10+ Rec TD Season*

In [29]:
pass_plays_df = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('td_team').isNotNull()) \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('td_team') == 'GB') \
                .groupby('td_player_name', 'td_player_id', 'season') \
                .agg(
                F.count('*').alias('Total Touchdowns')
                )\
                .filter(F.col('Total Touchdowns') >= 10) \
                .sort('Total Touchdowns', ascending=False) \
                .show()

+--------------+------------+------+----------------+
|td_player_name|td_player_id|season|Total Touchdowns|
+--------------+------------+------+----------------+
|       D.Adams|  00-0031381|  2020|              18|
|      J.Nelson|  00-0026176|  2011|              15|
|      J.Nelson|  00-0026176|  2016|              14|
|      Ja.Jones|  00-0025465|  2012|              13|
|      J.Nelson|  00-0026176|  2014|              13|
|       D.Adams|  00-0031381|  2018|              13|
|      J.Walker|  00-0021153|  2004|              12|
|    G.Jennings|  00-0024267|  2007|              12|
|    G.Jennings|  00-0024267|  2010|              12|
|        R.Cobb|  00-0028002|  2014|              12|
|       D.Adams|  00-0031381|  2016|              12|
|      R.Tonyan|  00-0033757|  2020|              11|
|       D.Adams|  00-0031381|  2021|              11|
|       D.Adams|  00-0031381|  2017|              10|
+--------------+------------+------+----------------+



### *What is a receiver on the Eagles with a 10+ Rec TD Season*

In [30]:
pass_plays_df = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('td_team').isNotNull()) \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('td_team') == 'PHI') \
                .groupby('td_player_name', 'td_player_id', 'season') \
                .agg(
                F.count('*').alias('Total Touchdowns')
                )\
                .filter(F.col('Total Touchdowns') >= 10) \
                .sort('Total Touchdowns', ascending=False) \
                .show()

+--------------+------------+------+----------------+
|td_player_name|td_player_id|season|Total Touchdowns|
+--------------+------------+------+----------------+
|       T.Owens|  00-0012478|  2004|              14|
|       A.Brown|  00-0035676|  2022|              11|
|      J.Maclin|  00-0026995|  2010|              10|
|      J.Maclin|  00-0026995|  2014|              10|
+--------------+------------+------+----------------+



### Player that has played for both Chiefs and Eagles

In [38]:
chiefs_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'KC') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

eagles_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'PHI') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'eagles_receiver_id') \
                .distinct() \


both = eagles_receivers.join(chiefs_receivers, eagles_receivers.eagles_receiver_id == chiefs_receivers.receiver_player_id, 'inner') \
        .show()

+------------------+------------------+--------------------+
|eagles_receiver_id|receiver_player_id|receiver_player_name|
+------------------+------------------+--------------------+
|        00-0027188|        00-0027188|              C.Hall|
|        00-0026995|        00-0026995|            J.Maclin|
|        00-0024324|        00-0024324|             J.Avant|
|        00-0027029|        00-0027029|             L.McCoy|
+------------------+------------------+--------------------+



### Player that has Played for both the Chiefs and Vikings

In [39]:
chiefs_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'KC') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

vikings_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'MIN') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'vikings_receiver_id') \
                .distinct() \


both = vikings_receivers.join(chiefs_receivers, vikings_receivers.vikings_receiver_id == chiefs_receivers.receiver_player_id, 'inner') \
        .show()

+-------------------+------------------+--------------------+
|vikings_receiver_id|receiver_player_id|receiver_player_name|
+-------------------+------------------+--------------------+
|         00-0000145|        00-0000145|         D.Alexander|
|         00-0013694|        00-0013694|        T.Richardson|
|         00-0013694|        00-0013694|       T. Richardson|
|         00-0000145|        00-0000145|        D. Alexander|
|         00-0020514|        00-0020514|           M.Bennett|
|         00-0022740|        00-0022740|             J.Allen|
|         00-0022144|        00-0022144|              B.Wade|
|         00-0032062|        00-0032062|              B.Bell|
|         00-0031376|        00-0031376|          J.McKinnon|
+-------------------+------------------+--------------------+



#### Players that have played for both the Chiefs and Packers

In [40]:
chiefs_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'KC') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

packers_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'GB') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'packers_receiver_id') \
                .distinct() \


both = packers_receivers.join(chiefs_receivers, packers_receivers.packers_receiver_id == chiefs_receivers.receiver_player_id, 'inner') \
        .show()

+-------------------+------------------+--------------------+
|packers_receiver_id|receiver_player_id|receiver_player_name|
+-------------------+------------------+--------------------+
|         00-0020525|        00-0020525|           R.Gardner|
|         00-0030280|        00-0030280|             K.Davis|
|         00-0031325|        00-0031325|           S.Watkins|
|         00-0034272|        00-0034272|  M.Valdes-Scantling|
+-------------------+------------------+--------------------+



#### Rams and Eagles

In [41]:
rams_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'LA') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

eagles_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'PHI') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'eagles_receiver_id') \
                .distinct() \



both = eagles_receivers.join(rams_receivers, eagles_receivers.eagles_receiver_id == rams_receivers.receiver_player_id, 'inner') \
        .show()

+------------------+------------------+--------------------+
|eagles_receiver_id|receiver_player_id|receiver_player_name|
+------------------+------------------+--------------------+
|        00-0009738|        00-0009738|               A.Lee|
|        00-0021209|        00-0021209|           L. Gordon|
|        00-0022001|        00-0022001|            K.Curtis|
|        00-0021209|        00-0021209|            L.Gordon|
|        00-0027700|        00-0027700|           M.Gilyard|
|        00-0023108|        00-0023108|             S.Spach|
|        00-0025438|        00-0025438|             S.Smith|
|        00-0026189|        00-0026189|           D.Jackson|
+------------------+------------------+--------------------+



#### Packers and Rams

In [42]:
rams_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'LA') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

packers_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'GB') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'packers_receiver_id') \
                .distinct() \




both = packers_receivers.join(rams_receivers, packers_receivers.packers_receiver_id == rams_receivers.receiver_player_id, 'inner') \
        .show()

+-------------------+------------------+--------------------+
|packers_receiver_id|receiver_player_id|receiver_player_name|
+-------------------+------------------+--------------------+
|         00-0020964|        00-0020964|           To.Fisher|
|         00-0023074|        00-0023074|              S.Gado|
|         00-0022421|        00-0022421|            R.Martin|
|         00-0027985|        00-0027985|         L.Kendricks|
|         00-0030525|        00-0030525|            T.Austin|
|         00-0027061|        00-0027061|              J.Cook|
|         00-0031325|        00-0031325|           S.Watkins|
+-------------------+------------------+--------------------+



#### Vikings and Rams

In [43]:
rams_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'LA') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id', 'receiver_player_name') \
                .distinct() \

vikings_receivers = spark_nfl_df \
                .filter(F.col('play_type') == 'pass') \
                .filter(F.col('season_type') == 'REG') \
                .filter(F.col('posteam') == 'MIN') \
                .filter(F.col('receiver_player_id').isNotNull()) \
                .select('receiver_player_id') \
                .withColumnRenamed('receiver_player_id', 'vikings_receiver_id') \
                .distinct() \




both = vikings_receivers.join(rams_receivers, vikings_receivers.vikings_receiver_id == rams_receivers.receiver_player_id, 'inner') \
        .show()

+-------------------+------------------+--------------------+
|vikings_receiver_id|receiver_player_id|receiver_player_name|
+-------------------+------------------+--------------------+
|         00-0022582|        00-0022582|             R.Owens|
|         00-0019721|        00-0019721|            T.Taylor|
|         00-0033246|        00-0033246|             J.Mundt|
|         00-0036414|        00-0036414|             C.Akers|
|         00-0034646|        00-0034646|            B.Powell|
+-------------------+------------------+--------------------+



#### Grid Results using options from PySpark Queries
#### ![Immaculate Grid Complete](https://raw.githubusercontent.com/jwolfe972/Spark-Projects/main/spark-learning-notebooks/images/nfl-imgs/immaculate_grid.png)

